In [ ]:
# For Google Colab: uncomment and run this cell to install dependencies
# !pip install rustworkx matplotlib numpy scipy

In [ ]:
# Core libraries
import random
import math
from math import factorial, exp
import numpy as np
from scipy.stats import binom, poisson
from scipy import special as sp
import matplotlib.pyplot as plt
from collections import defaultdict
import itertools
from itertools import combinations
from pathlib import Path

# Graph libraries
import rustworkx as rx
from rustworkx.visualization import mpl_draw

In [ ]:
# Visualization settings
np.set_printoptions(precision=2, suppress=True)

NS_PURPLE = "#8e44ad"  # Node color
NS_GREEN = "#2ecc71"   # Edge color
NS_ORANGE = "#FF9800"  # Highlight color

plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams.update({
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.edgecolor": "0.3",
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
})

# File paths
ROOT = Path.cwd()).parent if 'google.colab' not in str(get_ipython()) else Path('/content')
DATADIR = ROOT / "data"

In [ ]:
# Utility functions
def pprint(G):
    """Pretty print basic graph statistics."""
    print(f"Graph has {G.num_nodes()} nodes and {G.num_edges()} edges")

---

## 4.2 Power Laws and Scale-Free Networks

Many network quantities (like degree and centrality) exhibit **fat-tailed behavior** consistent with a power-law, meaning there is no characteristic "typical" scale. As a result, large events aren't anomalies—they're a predictable consequence of the distribution's *tail*.

### Discrete Formalism

For discrete degree values $k \in \{1, 2, 3, \ldots\}$:

$$p_k = C k^{-\gamma}$$

The constant $C$ is determined by the normalization condition:

$$C \sum_{k=1}^{\infty} k^{-\gamma} = 1$$

In [ ]:
gamma = 2
ks = list(range(1, 1000))

# Compute normalization constant by truncated sum
k_gamma_sum = sum(k**-gamma for k in ks)
C = 1.0 / k_gamma_sum

# Compute p_k values
pks = C * np.array([k**-gamma for k in ks])

print(f"C (truncated) = {C:.6f}")
print(f"Sum of p_k = {pks.sum():.4f}")

### Riemann Zeta Function

The normalization constant $C$ can be expressed in closed form using the **Riemann zeta function**:

$$C = \frac{1}{\zeta(\gamma)}$$

where $\zeta(\gamma) = \sum_{k=1}^{\infty} k^{-\gamma}$.

Thus the discrete power-law (zeta) distribution has the form:

$$p_k = \frac{k^{-\gamma}}{\zeta(\gamma)}$$

In [ ]:
def discrete_power_pmf(k, gamma=2):
    """Discrete power-law (zeta) probability mass function."""
    return (k ** -gamma) / sp.zeta(gamma, 1)

# Verify our manual calculation matches the zeta function
k_test = 3
manual = C * k_test**-gamma
zeta_based = discrete_power_pmf(k_test, gamma=2)

print(f"Manual calculation: p({k_test}) = {manual:.6f}")
print(f"Zeta-based:         p({k_test}) = {zeta_based:.6f}")
print(f"Match: {np.isclose(manual, zeta_based, rtol=0.01)}")

---

## 4.3 Hubs

The main difference between a random and a scale-free network comes in the **tail of the degree distribution**, representing the high-$k$ region of $p_k$.

| Property | Random Network | Scale-Free Network |
|----------|----------------|-------------------|
| Degree distribution | Poisson | Power-law |
| Tail behavior | Exponential decay | Fat tail |
| Hubs | Rare/absent | Common |
| Characteristic scale | Yes ($\langle k \rangle$) | No |

In [ ]:
# Compare specific probabilities
k = 5
gamma = 2.1
mu = 11  # average degree for Poisson

p_power = discrete_power_pmf(k, gamma=gamma)
p_poisson = poisson.pmf(k, mu=mu)

print(f"P(k={k}) for power-law (γ={gamma}): {p_power:.4f}")
print(f"P(k={k}) for Poisson (⟨k⟩={mu}):    {p_poisson:.4f}")

### Comparing Power-Law vs Poisson Distributions

In [ ]:
ks = np.arange(1, 50, dtype=float)
poisson_pks = poisson.pmf(ks, mu=11)
power_pks = discrete_power_pmf(ks, gamma=2.1)

plt.figure(figsize=(6, 4))
plt.plot(ks, power_pks, linewidth=3, color=NS_PURPLE, label='Power-law (γ=2.1)')
plt.plot(ks, poisson_pks, linewidth=3, color=NS_GREEN, label='Poisson (⟨k⟩=11)')

plt.xlim(0, 50)
plt.ylim(0, 0.15)
plt.xlabel("k")
plt.ylabel(r"$p_k$")
plt.title("Degree Distribution: Power-Law vs Poisson")
plt.legend()
plt.tight_layout()
plt.show()

The **power-law distribution** has a much heavier tail than the Poisson distribution. This means:

- In random networks, most nodes have degree close to $\langle k \rangle$
- In scale-free networks, there is significant probability of finding nodes with very high degree (**hubs**)

The term "scale-free" refers to the absence of a characteristic scale in the degree distribution.

---

## Summary

In this chapter, we explored the **scale-free property** of networks:

**4.2 Power Laws and Scale-Free Networks**
- Power-law degree distribution: $p_k = k^{-\gamma}/\zeta(\gamma)$
- No characteristic scale—distribution is self-similar
- Normalization via Riemann zeta function

**4.3 Hubs**
- Scale-free networks have fat-tailed degree distributions
- Hubs (high-degree nodes) are common, unlike in random networks
- Poisson distributions decay exponentially; power-laws decay slowly